02. DETECTIONS TRACKING

In [11]:
import os
os.environ["ONNXRUNTIME_EXECUTION_PROVIDERS"] = "[CUDAExecutionProvider]"

import sys
sys.path.append(os.getenv("PROJECT_PATH"))

from inference import get_model
import supervision as sv

ROBOFLOW_API_KEY = os.getenv("ROBOFLOW_API_KEY")

PLAYER_DETECTION_MODEL_ID = 'football-players-detection-3zvbc/2'
PLAYER_DETECTION_MODEL = get_model(PLAYER_DETECTION_MODEL_ID, ROBOFLOW_API_KEY)

FIELD_DETECTION_MODEL_ID = "football-field-detection-f07vi/14"
FIELD_DETECTION_MODEL = get_model(model_id=FIELD_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

SOURCE_VIDEO_PATH = "../../data/videos/3-fullvideo.mp4"

In [12]:
#Supervision annotators
from utils.pitchconfig import SoccerPitchConfiguration

CONFIG = SoccerPitchConfiguration()

ellipse_annotator = sv.EllipseAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']), #blue, pink, yellow
    thickness=2
)
label_annotator = sv.LabelAnnotator(
    color=sv.ColorPalette.from_hex(['#00BFFF', '#FF1493', '#FFD700']), #blue, pink, yellow
    text_color=sv.Color.from_hex('#000000'),
    text_position=sv.Position.BOTTOM_CENTER
)
triangle_annotator = sv.TriangleAnnotator(
    color=sv.Color.from_hex('#FFD700'),
    base=25,
    height=21,
    outline_thickness=1
)

#Supervision - virtualization
edge_annotator = sv.EdgeAnnotator(
    color=sv.Color.from_hex('#00BFFF'),
    thickness=2, edges=CONFIG.edges)
vertex_annotator = sv.VertexAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    radius=8)
vertex_annotator_2 = sv.VertexAnnotator(
    color=sv.Color.from_hex('#00BFFF'),
    radius=8)

In [13]:
#Team Assignment based on colours
#use of the SigLIP, UMAP, and KMeans combo
import supervision as sv
from tqdm import tqdm
from utils.teamclassifier import TeamClassifier

PLAYER_ID = 2
STRIDE = 20
frame_generator = sv.get_video_frames_generator(
    source_path=SOURCE_VIDEO_PATH, stride=STRIDE
)

crops = []
for frame in tqdm(frame_generator, desc="collecting crops"):
    result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.4)[0]
    detections = sv.Detections.from_inference(result)
    players_detections = detections[detections.class_id == PLAYER_ID]
    
    # Skip the frame if no players are detected
    if len(players_detections.xyxy) == 0:
        continue
    
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    crops += players_crops

# Ensure there are enough crops for clustering
if len(crops) < 2:
    print("Not enough player crops detected. Skipping team classification.")
else:
    team_classifier = TeamClassifier(device="cpu")
    team_classifier.fit(crops)
    
print(f"Number of players detected: {len(crops)}")

collecting crops: 137it [02:03,  1.11it/s]
Embedding extraction: 91it [16:10, 10.67s/it]


Number of players detected: 2891


In [14]:
#Frame collection for training
from tqdm import tqdm

frames = []

for frame_index, frame in enumerate(tqdm(sv.get_video_frames_generator(SOURCE_VIDEO_PATH), desc="Processing...")):
    if frame_index % 5 == 0: #default: 5
        frames.append(frame)

print(f"Total frames collected: {len(frames)}")

Processing...: 2725it [00:25, 105.50it/s]


Total frames collected: 545


In [15]:
#Full tracking
import supervision as sv
import numpy as np
from utils.viewtransformer import ViewTransformer
from utils.drawpitch import draw_pitch, draw_points_on_pitch
from utils.resolveteamgk import resolve_goalkeepers_team_id

BALL_ID = 0
PLAYER_ID = 2
GOALKEEPER_ID = 1
REFEREE_ID = 3

pitch_frames = []

ball_coords = []
gk_t1_coords = []
gk_t2_coords = []

player_coords_per_id = dict()
player_team_per_id = {}

# Tracker
tracker = sv.ByteTrack()

for frame_index, frame in enumerate(tqdm(frames, desc="Processing...")):
    tqdm.write(f"... frame {frame_index}: ")
    
    # ball and players detections
    result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    detections = sv.Detections.from_inference(result)

    ball_detections = detections[detections.class_id == BALL_ID]
    ball_detections.xyxy = sv.pad_boxes(xyxy=ball_detections.xyxy, px=10)
    
    all_detections = detections[detections.class_id != BALL_ID]
    all_detections = detections[detections.class_id != REFEREE_ID]
    all_detections = all_detections.with_nms(threshold=0.3, class_agnostic=True)
    all_detections = tracker.update_with_detections(detections=all_detections)

    goalkeepers_detections = all_detections[all_detections.class_id == GOALKEEPER_ID]
    players_detections = all_detections[all_detections.class_id == PLAYER_ID]

    # team assignment
    players_crops = [sv.crop_image(frame, xyxy) for xyxy in players_detections.xyxy]
    players_detections.class_id = team_classifier.predict(players_crops)

    if len(goalkeepers_detections.xyxy) > 0:
        goalkeepers_detections.class_id = resolve_goalkeepers_team_id(
            players_detections, goalkeepers_detections)
    else:
        #if no gk detected no need to resolve team id
        goalkeepers_detections.class_id = np.array([])
    
    all_detections = sv.Detections.merge([players_detections, goalkeepers_detections])

    # frame visualization
    labels = []
    for idx, tracker_id in enumerate(all_detections.tracker_id):
        class_id = all_detections.class_id[idx]
        if tracker_id in player_team_per_id:
            team = player_team_per_id[tracker_id]
        else:
            team = class_id
        labels.append(f"{tracker_id}_{team}")

    all_detections.class_id = all_detections.class_id.astype(int)

    annotated_frame = frame.copy()
    annotated_frame = ellipse_annotator.annotate(
        scene=annotated_frame,
        detections=all_detections)
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=all_detections,
        labels=labels)
    annotated_frame = triangle_annotator.annotate(
        scene=annotated_frame,
        detections=ball_detections)

    pitch_frames.append(annotated_frame.copy())
    
    players_detections = sv.Detections.merge([
        players_detections, goalkeepers_detections
    ])
    
    # Detect pitch key points
    result = FIELD_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    key_points = sv.KeyPoints.from_inference(result)
    
    # Ensure key_points is not None and contains valid data
    if key_points is None or key_points.confidence is None:
        print(f"Skipping frame {frame_index} due to missing key points.")
        continue  # Skip this frame and move to the next one

    # project ball, players and gk on pitch
    filter = key_points.confidence[0] > 0.5
    frame_reference_points = key_points.xy[0][filter]
    pitch_reference_points = np.array(CONFIG.vertices)[filter]

    # Ensure there are at least 4 points for homography calculation
    if len(frame_reference_points) < 4 or len(pitch_reference_points) < 4:
        print(f"Skipping frame due to insufficient keypoints: {len(frame_reference_points)} found.")
        continue  # Skip this frame and move to the next one

    # Proceed with homography calculation
    transformer = ViewTransformer(
        source = frame_reference_points,
        target = pitch_reference_points
    )

    frame_ball_xy = ball_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_ball_xy = transformer.transform_points(points=frame_ball_xy)

    players_xy = players_detections.get_anchors_coordinates(sv.Position.BOTTOM_CENTER)
    pitch_players_xy = transformer.transform_points(points=players_xy)
    
    # --- Save player coordinates per ID in pitch space ---
    for idx, tracker_id in enumerate(players_detections.tracker_id):
        if tracker_id is not None:
            if tracker_id not in player_coords_per_id:
                player_coords_per_id[tracker_id] = []
                # Save the team assignment for this tracker_id
                player_team_per_id[tracker_id] = int(players_detections.class_id[idx])
            x, y = pitch_players_xy[idx]
            player_coords_per_id[tracker_id].append((frame_index, x, y))

    # --- Save ball coordinates in pitch space ---
    if len(pitch_ball_xy) > 0:
        x, y = pitch_ball_xy[0]
        ball_coords.append((frame_index, x, y))
    else:
        ball_coords.append((frame_index, None, None))

... frame 0: 


Processing...:   0%|          | 0/545 [00:00<?, ?it/s]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:10, 10.41s/it]


... frame 1: 


Processing...:   0%|          | 1/545 [00:14<2:15:08, 14.91s/it]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:07,  7.11s/it]


... frame 2: 


...

Processing...:  99%|█████████▉| 542/545 [1:19:53<00:28,  9.40s/it]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:06,  6.93s/it]


... frame 543: 


Processing...: 100%|█████████▉| 543/545 [1:20:02<00:18,  9.42s/it]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:06,  6.46s/it]


... frame 544: 


Processing...: 100%|█████████▉| 544/545 [1:20:11<00:09,  9.26s/it]
Embedding extraction: 0it [00:00, ?it/s]
Embedding extraction: 1it [00:06,  6.59s/it]
Processing...: 100%|██████████| 545/545 [1:20:20<00:00,  8.85s/it]


In [16]:
#Save frames with detections to folder
import cv2
import numpy as np
import os

video_name = os.path.splitext(os.path.basename(SOURCE_VIDEO_PATH))[0]

frames_len = len(frames)

output_dir = f"../../data/frames/"
os.makedirs(output_dir, exist_ok=True)

for idx, frame in enumerate(pitch_frames):
    frame_path = os.path.join(output_dir, f"frame_{idx:04d}.png")
    
    cv2.imwrite(frame_path,np.array(frame))
    
print(f"frames saved to {output_dir}")

frames saved to ../../data/frames/


In [17]:
#Save coordinates per frame to CSV
import csv
import numpy as np
import os

video_name = os.path.splitext(os.path.basename(SOURCE_VIDEO_PATH))[0]

output_csv_path = f"../../data/raw-coords/{video_name}.csv" 
all_ids = sorted(player_coords_per_id.keys())

# Build lookups
frame_lookup = {}
for tracker_id, coords in player_coords_per_id.items():
    for frame_index, x, y in coords:
        if frame_index not in frame_lookup:
            frame_lookup[frame_index] = {}
        frame_lookup[frame_index][tracker_id] = [x, y]

ball_lookup = {f: [x, y] for f, x, y in ball_coords}

header = (
    ["frame_index", "ball"] +
    [f"id_{id}_team_{player_team_per_id[id]}" for id in all_ids]
)

all_frames = sorted(set(
    list(frame_lookup.keys()) +
    list(ball_lookup.keys())
))

with open(output_csv_path, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    for frame_index in all_frames:
        row = [frame_index]
        # Ball
        ball = ball_lookup.get(frame_index, None)
        if ball and None not in ball:
            row.append(f"{ball[0]:.2f},{ball[1]:.2f}")
        else:
            row.append("")
        # Players by id
        for id in all_ids:
            coord = frame_lookup.get(frame_index, {}).get(id, None)
            if coord and None not in coord:
                row.append(f"{coord[0]:.2f},{coord[1]:.2f}")
            else:
                row.append("")
        writer.writerow(row)

print(f"Coordinates per frame saved to {output_csv_path}")

Coordinates per frame saved to ../../data/raw-coords/3-fullvideo.csv
